In [1]:
# ********** Import Statements ************************************************************ #
import sys
import spotipy
import spotipy.util as util

import pandas as pd
import json
import math

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import datasets, linear_model

pd.set_option('display.max_rows', 500)

In [2]:
# ********** Define Intro Values ************************************************************ #
# Loading in the client id and secret.
#client_info = json.load(open('/Users/cassielebauer/Documents/Projects/secret credentials/sameify_creds.json'))

# Spotify user IDs for testing: these are public information.
user_ids = {'Cassie':'1242091675',
            'Caity':'12166088827',
            'Ben':'yaojxcyh5tczzlln0ygm9sfla',
            'Alex':'12130992491',
            'Jason':'jason.albert'}

In [3]:
class Sameify:
    # Scopes for initialization; these are for the Spotify API
    redirect_uri = 'http://localhost:1410/'
    client_info = json.load(open('/Users/cassielebauer/Documents/Projects/secret credentials/sameify_creds.json'))
    scopes = ['user-read-recently-played',
              'user-top-read',
              'playlist-read-collaborative',
              'user-library-read',
              'playlist-read-private',
              'user-read-private',
              'playlist-modify-public']

    def __init__(self, user_id, read_only=True):
        # Declare valuesl
        self.id = user_id
        self.client_id = self.client_info['client_id']
        self.client_secret = self.client_info['client_secret']
        
        # Connect to Spotify
        self.sp = self.spotify_connect()
        
        # Pull relevant main values
        self.name = self.sp.user(self.id)['display_name']
        self.playlists = self.get_all_playlists()
        self.personal_playlists = self.get_personal_playlists(self.playlists)
        self.personal_tracks = self.get_all_playlist_tracks(self.personal_playlists[0:2])

    # Connects to Spotify
    def spotify_connect(self):
        token = util.prompt_for_user_token(self.id,
                                            self.scopes,
                                            self.client_id,
                                            self.client_secret,
                                            self.redirect_uri)

        sp = spotipy.Spotify(auth=token)
        return sp
    
    #def overstep_limit(self, item_id):

    def get_playlist_id(self, playlist_name):
        return self.playlists.loc[self.playlists['playlist_name'] == playlist_name,'playlist_id'].values[0]
    
    def get_playlist_name(self, playlist_id):
        return self.playlists.loc[self.playlists['playlist_id'] == playlist_id,'playlist_name'].values[0]
    
    def get_all_playlist_tracks(self, list_of_playlists):
        num_tracks = sum(list_of_playlists['num_tracks'])
        input_txt = f'This may load up to {num_tracks} songs, which will take approximately {num_tracks//9} seconds. Please type Y to continue'
        cont = input(input_txt)
        if cont.upper() == 'Y':
            tracks_df = pd.DataFrame()

            for ix in range(len(list_of_playlists)):
                plist_tracks = self.get_playlist_tracks(list_of_playlists.loc[ix,'playlist_id'])
                tracks_df = tracks_df.append(plist_tracks)
            return tracks_df
        else:
            pass
            
    def get_playlist_tracks(self, playlist_id):
        ret_df = pd.DataFrame()
        
        playlist = self.sp.user_playlist_tracks(self.id, playlist_id)
        playlist_name = self.get_playlist_name(playlist_id)
        tracks = playlist['items']
        
        while playlist['next']:
            playlist = self.sp.next(playlist)
            tracks.extend(playlist['items'])
            
        for track in tracks:
            x = track['track']
            data = {'playlist_id'  : playlist_id,
                    'playlist_name': playlist_name,
                    'track_name'   : x['name'],
                    'track_id'     : x['id'], 
                    'track_artist' : x['artists'][0]['name'], #[artist['name'] for artist in x['artists']]
                    'added_by'     : track['added_by']['id'],
                    'added_at'     : track['added_at']
                   }
            data_df = pd.DataFrame([data])
            song_deets = pd.DataFrame(self.sp.audio_features(data['track_id']))
            
            full_data = pd.concat([data_df, song_deets],axis=1)
            
            ret_df = ret_df.append(full_data)
        ret_df.reset_index(drop=True, inplace=True)
        return ret_df
            
        
    def get_all_playlists(self):
        playlists = pd.DataFrame()
        
        total = self.sp.user_playlists(self.id)['total'] #number of playlists
        limit = 50 # pull at a time
        
        for x in range(0, total, limit): # for each set of playlists
            new_playlist = self.sp.user_playlists(self.id, limit=limit, offset=x)
            for x in new_playlist['items']:
                items = {'playlist_name': x['name'],
                         'playlist_id'  : x['id'],
                         'owner_name'   : x['owner']['display_name'],
                         'owner_id'     : x['owner']['id'],
                         'num_tracks'   : x['tracks']['total'],
                         'collaborative': x['collaborative'],
                         }
                playlists = playlists.append(pd.DataFrame([items]))
        
        playlists.reset_index(drop=True, inplace=True)
        return playlists
     
    def remove_long_playlists(self, playlists):        
        req_text = '''The following playlists have over 100 tracks. Do you want to exclude any of these from the program? 
If so, please list the index numbers in a comma separated list. (e.g. "1,2,5,10").
For all playlists, please type "all".'''
        print(playlists[playlists['num_tracks']>=100][['playlist_name','num_tracks']])
        
        indices = input(req_text)
        if indices == 'all':
            print(playlists[['playlist_name','num_tracks']])
            req_text_2 = 'Please list the index numbers of the playlists to remove in a comma separated list. (e.g. "1,2,5,10")'
            indices = input(req_text_2)
        
        indices_to_exclude = [int(x) for x in indices.split(',')] if indices != '' else []

        indices_to_include = [x for x in playlists.index if x not in indices_to_exclude]
        playlists_to_use = playlists.iloc[indices_to_include,:].reset_index(drop=True)
        return playlists_to_use
    
    def get_personal_playlists(self, playlists):
        collab_filter = (playlists['collaborative'] == True) & (playlists['owner_name'] != self.name)
        personal_filter = playlists['owner_name'] == self.name
        personal_playlists = playlists[collab_filter | personal_filter].reset_index(drop=True)
        playlists_to_use = self.remove_long_playlists(personal_playlists)
        return playlists_to_use
 

In [4]:
# 1, 2, 3, 203, 204, 205, 206

In [5]:
cassie_acct = Sameify(user_ids['Cassie'],read_only=False)

                             playlist_name  num_tracks
9                         Recent old faves         101
12              All Songs (as of 01/11/22)        2268
14                 Discover Weekly Archive        1183
15                   Release Radar Archive         978
25                      The Nechromanticon         125
56                          Teenage Cassie         228
58   It's LIT! Essential HHM Playlist 2021         148
150                     Discovered & Loved         149
151                         Songs Saved v1         113
231               Release Radar Archive v1        4987
232             Discover Weekly Archive v2        4385
234                Discover Weekly Archive         502
235                  Release Radar Archive         505
The following playlists have over 100 tracks. Do you want to exclude any of these from the program? 
If so, please list the index numbers in a comma separated list. (e.g. "1,2,5,10").
For all playlists, please type "all".
This may

In [30]:
song_about_rain_dict = cassie_acct.sp.search('Song About Rain Judah S. Goldman')['tracks']['items'][0]

In [40]:
cassie_acct.sp.artist('0OcpF3dRDGb8cJneTvXlot')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/0OcpF3dRDGb8cJneTvXlot'},
 'followers': {'href': None, 'total': 6},
 'genres': [],
 'href': 'https://api.spotify.com/v1/artists/0OcpF3dRDGb8cJneTvXlot',
 'id': '0OcpF3dRDGb8cJneTvXlot',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b273354977cd6b5fe61a21586361',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/ab67616d00001e02354977cd6b5fe61a21586361',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67616d00004851354977cd6b5fe61a21586361',
   'width': 64}],
 'name': 'José D. Cruz',
 'popularity': 0,
 'type': 'artist',
 'uri': 'spotify:artist:0OcpF3dRDGb8cJneTvXlot'}

In [39]:
cassie_acct.sp.search("Jose D Cruz")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Jose+D+Cruz&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/56TkPi7rpmU8jTpkcK7FY3'},
       'href': 'https://api.spotify.com/v1/artists/56TkPi7rpmU8jTpkcK7FY3',
       'id': '56TkPi7rpmU8jTpkcK7FY3',
       'name': 'Beth Carvalho',
       'type': 'artist',
       'uri': 'spotify:artist:56TkPi7rpmU8jTpkcK7FY3'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CY',
      'CZ',
      'DE',
      'DJ',
  

In [30]:
cassie_acct.sp.current_user_recently_played(limit=50)['next']

'https://api.spotify.com/v1/me/player/recently-played?before=1623870310702&limit=50'

In [38]:
st_use = st[60:-9]

In [40]:
cassie_acct.sp.current_user_recently_played(limit=50, after=st_use)

{'items': [{'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0ZHPrnImGh4re3TbSNkoZl'},
       'href': 'https://api.spotify.com/v1/artists/0ZHPrnImGh4re3TbSNkoZl',
       'id': '0ZHPrnImGh4re3TbSNkoZl',
       'name': 'Beret',
       'type': 'artist',
       'uri': 'spotify:artist:0ZHPrnImGh4re3TbSNkoZl'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/5C4PDR4LnhZTbVnKWXuDKD'},
       'href': 'https://api.spotify.com/v1/artists/5C4PDR4LnhZTbVnKWXuDKD',
       'id': '5C4PDR4LnhZTbVnKWXuDKD',
       'name': 'Morat',
       'type': 'artist',
       'uri': 'spotify:artist:5C4PDR4LnhZTbVnKWXuDKD'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
     

In [31]:
st = 'https://api.spotify.com/v1/me/player/recently-played?before=1623870310702&limit=50'

In [67]:
cassie_acct.sp.current_user_recently_played(before="1624138668696")['cursors']

{'Affection',
 'All',
 'As the World Caves In',
 'Astronaut In The Ocean',
 'Blurred Lines',
 'Brand New Day',
 'CUANDO TE FUISTE',
 'Corazón Sin Vida',
 'Corner of Queen',
 'DOSIS',
 'Elena',
 'Flowers',
 'Get Out',
 'Grass',
 'Happy Now',
 "International Super Spy (There's One Thing You Can Expect from Me and That's the Unexpected)",
 'Intro',
 'Keyframe',
 'No Matter What (feat. Rebecca Sugar & Jeff Liu) - Demo',
 'Pompeii - Remix',
 'Qué Tienes Tú (feat. Jesús de Reik & Mau y Ricky)',
 'Snakes & Smoke',
 'Sorry',
 'Soy de Volar',
 'The Little Dream',
 "There's No Such Thing as Happily Ever After",
 'Us',
 'We Are the Crystal Gems (From "Steven Universe") [Orchestrated]',
 'calles',
 'good 4 u',
 'haiku',
 'oneofone_rwrk',
 'sincerely, yours'}

In [68]:
set(x['track']['name'] for x in cassie_acct.sp.current_user_recently_played(after="1623870310702")['items'])

{'Affection',
 'All',
 'As the World Caves In',
 'Astronaut In The Ocean',
 'Blurred Lines',
 'Brand New Day',
 'CUANDO TE FUISTE',
 'Corazón Sin Vida',
 'Corner of Queen',
 'DOSIS',
 'Elena',
 'Flowers',
 'Get Out',
 'Grass',
 'Happy Now',
 "International Super Spy (There's One Thing You Can Expect from Me and That's the Unexpected)",
 'Intro',
 'Keyframe',
 'No Matter What (feat. Rebecca Sugar & Jeff Liu) - Demo',
 'Pompeii - Remix',
 'Porfa no te vayas',
 'Qué Tienes Tú (feat. Jesús de Reik & Mau y Ricky)',
 'Snakes & Smoke',
 'Sorry',
 'Soy de Volar',
 'The Little Dream',
 "There's No Such Thing as Happily Ever After",
 'Us',
 'calles',
 'good 4 u',
 'haiku',
 'oneofone_rwrk',
 'sincerely, yours'}

In [ ]:
set(x['track']['name'] for x in cassie_acct.sp.current_user_recently_played(limit=50, before='1624138668696')['items'])

In [ ]:
{'after': '1624138668696', 'before': '1623870310702'}

In [54]:
# only works for SIGNED IN USER -> recently played songs.
set(x['track']['name'] for x in cassie_acct.sp.current_user_recently_played()['items'])

{'Affection',
 'All',
 'As the World Caves In',
 'Astronaut In The Ocean',
 'Blurred Lines',
 'Brand New Day',
 'CUANDO TE FUISTE',
 'Corazón Sin Vida',
 'Corner of Queen',
 'DOSIS',
 'Elena',
 'Flowers',
 'Get Out',
 'Grass',
 'Happy Now',
 "International Super Spy (There's One Thing You Can Expect from Me and That's the Unexpected)",
 'Intro',
 'Keyframe',
 'No Matter What (feat. Rebecca Sugar & Jeff Liu) - Demo',
 'Pompeii - Remix',
 'Porfa no te vayas',
 'Qué Tienes Tú (feat. Jesús de Reik & Mau y Ricky)',
 'Snakes & Smoke',
 'Sorry',
 'Soy de Volar',
 'The Little Dream',
 "There's No Such Thing as Happily Ever After",
 'Us',
 'We Are the Crystal Gems (From "Steven Universe") [Orchestrated]',
 'calles',
 'good 4 u',
 'haiku',
 'oneofone_rwrk',
 'sincerely, yours'}

In [13]:
ben_acct.sp.current_user_saved_tracks()

{'href': 'https://api.spotify.com/v1/me/tracks?offset=0&limit=20',
 'items': [{'added_at': '2021-06-17T18:17:03Z',
   'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1DHHaCy2ri351JypVkX9B0'},
       'href': 'https://api.spotify.com/v1/artists/1DHHaCy2ri351JypVkX9B0',
       'id': '1DHHaCy2ri351JypVkX9B0',
       'name': 'MADAX',
       'type': 'artist',
       'uri': 'spotify:artist:1DHHaCy2ri351JypVkX9B0'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK'

In [9]:
ben_acct = Sameify(user_ids['Ben'],read_only=True)

    playlist_name  num_tracks
2        Workout!         222
8   Golden Oldies         385
11        Writing         113
15        Running         613
The following playlists have over 100 tracks. Do you want to exclude any of these from the program? 
If so, please list the index numbers in a comma separated list. (e.g. "1,2,5,10").
For all playlists, please type "all".2,8,11,15
This may load up to 117 songs, which will take approximately 13 seconds. Please type Y to continuey


In [11]:
ben_acct.personal_tracks

,playlist_id,playlist_name,track_name,track_id,track_artist,added_by,added_at,danceability,energy,key,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,4AgXIKASWMr5rsjloJNPrP,Ben's Music Charcuterie,Murder! Murder!,2z2rcCzS0jBTya2SwQZIpW,American Murder Song,yaojxcyh5tczzlln0ygm9sfla,2020-11-23T20:42:14Z,0.704,0.690,5,...,0.6310,0.4920,111.935,audio_features,2z2rcCzS0jBTya2SwQZIpW,spotify:track:2z2rcCzS0jBTya2SwQZIpW,https://api.spotify.com/v1/tracks/2z2rcCzS0jBT...,https://api.spotify.com/v1/audio-analysis/2z2r...,219152,4
1,4AgXIKASWMr5rsjloJNPrP,Ben's Music Charcuterie,The Devil in Camp,6jfV9Y5nVFRzcHCxRgaGMC,American Murder Song,yaojxcyh5tczzlln0ygm9sfla,2020-11-23T20:42:45Z,0.633,0.755,4,...,0.3090,0.6960,75.041,audio_features,6jfV9Y5nVFRzcHCxRgaGMC,spotify:track:6jfV9Y5nVFRzcHCxRgaGMC,https://api.spotify.com/v1/tracks/6jfV9Y5nVFRz...,https://api.spotify.com/v1/audio-analysis/6jfV...,239013,4
2,4AgXIKASWMr5rsjloJNPrP,Ben's Music Charcuterie,Pumped up Kicks (Room 0),0Rx4vXgplon2ozMhds93V1,American Murder Song,yaojxcyh5tczzlln0ygm9sfla,2020-11-23T20:42:59Z,0.805,0.850,7,...,0.0673,0.8220,126.020,audio_features,0Rx4vXgplon2ozMhds93V1,spotify:track:0Rx4vXgplon2ozMhds93V1,https://api.spotify.com/v1/tracks/0Rx4vXgplon2...,https://api.spotify.com/v1/audio-analysis/0Rx4...,179000,4
3,4AgXIKASWMr5rsjloJNPrP,Ben's Music Charcuterie,Come to the Circus,4k9fR7zBEvwyN9j0GUEYbg,Circus Contraption,yaojxcyh5tczzlln0ygm9sfla,2020-11-23T20:43:15Z,0.517,0.633,9,...,0.1120,0.9630,165.285,audio_features,4k9fR7zBEvwyN9j0GUEYbg,spotify:track:4k9fR7zBEvwyN9j0GUEYbg,https://api.spotify.com/v1/tracks/4k9fR7zBEvwy...,https://api.spotify.com/v1/audio-analysis/4k9f...,170240,4
4,4AgXIKASWMr5rsjloJNPrP,Ben's Music Charcuterie,If I Told You Once,0Jkl79sElTWQnUr534STeJ,The Circus Contraption Band,yaojxcyh5tczzlln0ygm9sfla,2020-11-23T20:43:23Z,0.617,0.572,5,...,0.1890,0.4530,111.091,audio_features,0Jkl79sElTWQnUr534STeJ,spotify:track:0Jkl79sElTWQnUr534STeJ,https://api.spotify.com/v1/tracks/0Jkl79sElTWQ...,https://api.spotify.com/v1/audio-analysis/0Jkl...,276947,1
5,4AgXIKASWMr5rsjloJNPrP,Ben's Music Charcuterie,We're All Mad,5KUqeG9pyDA2YgI2o4YIwm,The Circus Contraption Band,yaojxcyh5tczzlln0ygm9sfla,2020-11-23T20:43:28Z,0.568,0.564,2,...,0.0916,0.5910,106.535,audio_features,5KUqeG9pyDA2YgI2o4YIwm,spotify:track:5KUqeG9pyDA2YgI2o4YIwm,https://api.spotify.com/v1/tracks/5KUqeG9pyDA2...,https://api.spotify.com/v1/audio-analysis/5KUq...,202987,4
6,4AgXIKASWMr5rsjloJNPrP,Ben's Music Charcuterie,A Bar In Amsterdam,0ytvlQNCJmSY6l6a840U5A,Katzenjammer,yaojxcyh5tczzlln0ygm9sfla,2020-11-23T20:43:39Z,0.589,0.799,7,...,0.3700,0.7130,143.954,audio_features,0ytvlQNCJmSY6l6a840U5A,spotify:track:0ytvlQNCJmSY6l6a840U5A,https://api.spotify.com/v1/tracks/0ytvlQNCJmSY...,https://api.spotify.com/v1/audio-analysis/0ytv...,178493,4
7,4AgXIKASWMr5rsjloJNPrP,Ben's Music Charcuterie,Rock-Paper-Scissors,5AErfeSAv0NPSivHpe12AU,Katzenjammer,yaojxcyh5tczzlln0ygm9sfla,2020-11-23T20:43:45Z,0.463,0.776,5,...,0.0633,0.8760,184.420,audio_features,5AErfeSAv0NPSivHpe12AU,spotify:track:5AErfeSAv0NPSivHpe12AU,https://api.spotify.com/v1/tracks/5AErfeSAv0NP...,https://api.spotify.com/v1/audio-analysis/5AEr...,208720,4
8,4AgXIKASWMr5rsjloJNPrP,Ben's Music Charcuterie,Soviet Trumpeter,46Cec5dxngueMCy0IBO3YJ,Katzenjammer,yaojxcyh5tczzlln0ygm9sfla,2020-11-23T20:43:49Z,0.392,0.394,1,...,0.1440,0.1210,76.450,audio_features,46Cec5dxngueMCy0IBO3YJ,spotify:track:46Cec5dxngueMCy0IBO3YJ,https://api.spotify.com/v1/tracks/46Cec5dxngue...,https://api.spotify.com/v1/audio-analysis/46Ce...,251947,4
9,4AgXIKASWMr5rsjloJNPrP,Ben's Music Charcuterie,Fear Of The Dark,3NvI1qSyQoFlsXg97jVVNM,Van Canto,yaojxcyh5tczzlln0ygm9sfla,2020-11-23T20:44:14Z,0.321,0.940,4,...,0.1770,0.2550,112.506,audio_features,3NvI1qSyQoFlsXg97jVVNM,spotify:track:3NvI1qSyQoFlsXg97jVVNM,https://api.spotify.com/v1/tracks/3NvI1qSyQoFl...,https://api.spotify.com/v1/audio-analysis/3NvI...,429387,4


In [5]:
cassie_acct.

In [ ]:
test_df = ben_acct.personal_playlists[0:2]

In [ ]:
ben_acct.personal_playlists[0:2]

In [ ]:
ben_acct.playlists

In [ ]:
cassie_acct.personal_playlists['num_tracks'].sum()

In [ ]:
ben_acct.personal_playlists['num_tracks'].sum()

In [ ]:
collab_filter = (ben_acct.playlists['collaborative'] == True) & (ben_acct.playlists['owner_name'] != self.name)
personal_filter = ben_acct.playlists['owner_name'] == self.name

In [ ]:
ben_acct.name

In [ ]:
import timeit

In [ ]:
start = timeit.default_timer()
xx = ben_acct.get_all_playlist_tracks(ben_acct.personal_playlists[0:2])
end = timeit.default_timer()
diff = end - start
print(f'Timer: {diff} sec for {len(xx)} entries')

In [ ]:
ben_acct.playlists

In [ ]:
ben_acct.id, ben_acct.name